# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

## Prepare Environment

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  9 00:41:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
!pip install datasets>=2.6.1
!pip install transformers
#!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 110.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB

In [ ]:
import os

# Run to fix bug of Seq2SeqTrainer not being found
# os._exit(00)

In [ ]:
from huggingface_hub import notebook_login

# Link HuggingFace in order to download dataset or to push model to repository
notebook_login()

# Feature Pipeline

## Load Dataset

In [ ]:
import os
import regex as re
from os.path import isfile, join
from datasets import concatenate_datasets, Dataset, load_dataset, DatasetDict
from google.colab import drive

DOWNLOAD = False

# Mount the drive
drive.mount('/content/drive')

# Create Dataset Dictionary
common_voice = DatasetDict()

# Get the arrow files storing training and testing data
r_train = re.compile("data-000[0-1][0-9]-of-00042.arrow")
arrow_files_train = list(filter(r_train.match, os.listdir("/content/drive/MyDrive/common_voice/train")))
r_test = re.compile("data-0000[0-1]-of-00002.arrow")
arrow_files_test = list(filter(r_test.match, os.listdir("/content/drive/MyDrive/common_voice/test")))

# Concatenate stored arrow files and assign as training and test data
if not DOWNLOAD:
  common_voice["train"] = concatenate_datasets([Dataset.from_file(join("/content/drive/MyDrive/common_voice/train", arrow_file)) for arrow_file in arrow_files_train]).select(range(10000))
  common_voice["test"] = concatenate_datasets([Dataset.from_file(join("/content/drive/MyDrive/common_voice/test", arrow_file)) for arrow_file in arrow_files_test]).select(range(5000))

# If unprocessed dataset should be downloaded from HuggingFace
if DOWNLOAD:
  common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "de", split="train+validation", use_auth_token=True)
  common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "de", split="test", use_auth_token=True)


Mounted at /content/drive


In [ ]:
# Run code cell if loaded dataset should be pushed and stored to Drive
output_dir = "/content/drive/MyDrive/common_voice"
os.makedirs(output_dir, exist_ok=True)
common_voice.save_to_disk(output_dir)

## Prepare Dataset

In [ ]:
# Prepare dataset for use with Whisper
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
from datasets import Audio

# Remove unneccessary features
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

# Change sampling frequency from 48kHz to 16kHz prior to using it for Whisper
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print(common_voice)
print(common_voice["train"][0])

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5000
    })
})


In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)
print(common_voice["train"][0])

In [ ]:
# Run code cell to push and store processed features to Drive

# Mount the Drive if not mounted
drive.mount('/content/drive')

# Processed features stored in 'features' folder
output_dir = "/content/drive/MyDrive/common_voice_features"
os.makedirs(output_dir, exist_ok=True)

# Save to Drive
common_voice.save_to_disk(output_dir)